# 05. Selector Group Chat

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 참가자들이 차례로 다른 모든 멤버에게 메시지를 방송하는 팀을 구현합니다. 생성 모델(예: LLM)이 공유된 컨텍스트를 기반으로 다음 발언자를 선택하여 동적이고 컨텍스트 인식 협업을 가능하게 합니다.

주요 기능:

- 모델 기반 발언자 선택
- 구성 가능한 참가자 역할 및 설명
- 동일한 발언자의 연속 발언 방지 (선택 사항)
- 사용자 정의 선택 프롬프트
- 기본 모델 기반 선택을 재정의하는 사용자 정의 선택 함수
- 모델을 사용하여 선택할 에이전트 세트를 좁히는 사용자 정의 후보 함수

```{note}
{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 고수준 API입니다. 더 많은 제어와 커스터마이징이 필요한 경우, 코어 API 문서의 [그룹 채팅 패턴](../core-user-guide/design-patterns/group-chat.ipynb)을 참조하여 자체 그룹 채팅 로직을 구현하세요.
```

## 어떻게 작동하나요?

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`와 유사한 그룹 채팅이지만, 모델 기반 다음 발언자 선택 메커니즘을 사용합니다.
팀이 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` 또는 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`을 통해 작업을 받으면, 다음 단계가 실행됩니다:

1. 팀은 대화 기록과 참가자들의 {py:attr}`~autogen_agentchat.base.ChatAgent.name` 및 {py:attr}`~autogen_agentchat.base.ChatAgent.description` 속성을 포함한 현재 대화 컨텍스트를 분석하여 모델을 사용해 다음 발언자를 결정합니다. 기본적으로 팀은 사용 가능한 에이전트가 하나뿐이 아닌 이상 동일한 발언자를 연속으로 선택하지 않습니다. 이는 `allow_repeated_speaker=True`로 설정하여 변경할 수 있습니다. 사용자 정의 선택 함수를 제공하여 모델을 재정의할 수도 있습니다.
2. 팀은 선택된 발언자 에이전트에게 응답을 요청하고, 이는 다른 모든 참가자에게 **방송**됩니다.
3. 대화가 끝나야 하는지 종료 조건을 확인하고, 그렇지 않다면 1단계부터 과정을 반복합니다.
4. 대화가 끝나면 팀은 이 작업의 대화 기록을 포함한 {py:class}`~autogen_agentchat.base.TaskResult`를 반환합니다.

팀이 작업을 완료하면 대화 컨텍스트는 팀과 모든 참가자에게 유지되므로, 다음 작업은 이전 대화 컨텍스트에서 계속할 수 있습니다.
{py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset`을 호출하여 대화 컨텍스트를 재설정할 수 있습니다.

이 섹션에서는 웹 검색 및 데이터 분석 작업의 간단한 예제로 {py:class}`~autogen_agentchat.teams.SelectorGroupChat`를 사용하는 방법을 보여드리겠습니다.

## 예제: 웹 검색/분석

In [6]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

## 1. 필수 라이브러리 Import

SelectorGroupChat 구현에 필요한 핵심 라이브러리들을 import합니다:
- **에이전트 클래스**: `AssistantAgent`, `UserProxyAgent`
- **종료 조건**: `MaxMessageTermination`, `TextMentionTermination`  
- **팀 관리**: `SelectorGroupChat`
- **UI**: `Console` (대화형 인터페이스)
- **메시지 타입**: 타입 힌팅용 클래스들

### 에이전트

![선택자 그룹 채팅](images/selector-group-chat.svg)

이 시스템은 세 개의 전문 에이전트를 사용합니다:

- **계획 에이전트**: 복잡한 작업을 관리 가능한 하위 작업으로 나누는 전략적 조정자입니다.
- **웹 검색 에이전트**: `search_web_tool`과 인터페이스하는 정보 검색 전문가입니다.
- **데이터 분석 에이전트**: `percentage_change_tool`을 갖춘 계산 수행 전문 에이전트입니다.

`search_web_tool`과 `percentage_change_tool`은 에이전트가 작업을 수행하는 데 사용할 수 있는 외부 도구입니다.

In [2]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:
        우도니스 하슬렘: 844점
        드웨인 웨이드: 1397점
        제임스 포시: 550점
        ...
        """
    elif "2007-2008" in query:
        return "2007-2008시즌 마이애미 히트에서 드웨인 웨이드가 기록한 총 리바운드 수는 214개입니다."
    elif "2008-2009" in query:
        return "2008-2009 시즌 마이애미 히트에서 드웨인 웨이드가 기록한 총 리바운드 수는 398개입니다."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

## 2. 도구 함수 정의 (Mock API)

실제 API 대신 시연용 모의(Mock) 도구 함수들을 정의합니다:

### 🔍 search_web_tool
- **목적**: 웹 검색을 시뮬레이션하는 함수
- **기능**: 특정 NBA 시즌 데이터를 쿼리에 따라 반환
- **실제 사용**: 실제 구현에서는 Google Search API 등을 활용

### 📊 percentage_change_tool  
- **목적**: 백분율 변화율 계산
- **공식**: `((end - start) / start) * 100`
- **용도**: 통계 데이터 분석에 활용

{py:class}`~autogen_agentchat.agents.AssistantAgent` 클래스를 사용하여 전문 에이전트를 생성해 보겠습니다.
에이전트의 {py:attr}`~autogen_agentchat.base.ChatAgent.name` 및 {py:attr}`~autogen_agentchat.base.ChatAgent.description` 속성이 모델에서 다음 발언자를 결정하는 데 사용되므로, 의미있는 이름과 설명을 제공하는 것이 좋습니다.

In [3]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv(override=True)

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

model_client = azure_openai_chat_completion_client


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

## 3. Azure OpenAI 클라이언트 설정

Azure OpenAI 서비스와의 연동을 위한 클라이언트를 설정합니다:

### 🔑 환경 변수 로딩
- `.env` 파일에서 API 키와 엔드포인트 정보 로딩
- `override=True`: 기존 환경 변수 덮어쓰기

### 🌐 클라이언트 초기화
- **model**: 사용할 Azure OpenAI 모델 (예: GPT-4)
- **azure_endpoint**: Azure OpenAI 리소스 엔드포인트
- **api_version**: API 버전 (예: "2024-02-15-preview")
- **api_key**: 인증용 API 키

```{note}
기본적으로 {py:class}`~autogen_agentchat.agents.AssistantAgent`는 도구 출력을 응답으로 반환합니다. 도구가 자연어 형식으로 잘 구성된 문자열을 반환하지 않는 경우, 에이전트를 생성할 때 `reflect_on_tool_use=True`를 설정하여 에이전트 내에서 반성 단계를 추가할 수 있습니다. 이를 통해 에이전트가 도구 출력을 반성하고 자연어 응답을 제공할 수 있습니다.
```

### 워크플로우

1. 작업이 {py:class}`~autogen_agentchat.teams.SelectorGroupChat`에 의해 수신되면, 에이전트 설명을 기반으로 초기 작업을 처리할 가장 적절한 에이전트(일반적으로 계획 에이전트)를 선택합니다.

2. **계획 에이전트**는 작업을 분석하고 하위 작업으로 나누어 다음 형식을 사용하여 각각을 가장 적절한 에이전트에 할당합니다:
   `<에이전트> : <작업>`

3. 대화 컨텍스트와 에이전트 설명을 기반으로 {py:class}`~autogen_agent.teams.SelectorGroupChat` 관리자가 할당된 하위 작업을 처리할 다음 에이전트를 동적으로 선택합니다.

4. **웹 검색 에이전트**는 한 번에 하나씩 검색을 수행하고 결과를 공유 대화 기록에 저장합니다.

5. **데이터 분석자**는 선택되었을 때 사용 가능한 계산 도구를 사용하여 수집된 정보를 처리합니다.

6. 다음 중 하나가 발생할 때까지 에이전트가 동적으로 선택되면서 워크플로우가 계속됩니다:
   - 계획 에이전트가 모든 하위 작업이 완료되었다고 판단하고 "TERMINATE"를 보냄
   - 대안적인 종료 조건이 충족됨 (예: 최대 메시지 수)

에이전트를 정의할 때 다음에 선택할 에이전트를 결정하는 데 사용되므로 도움이 되는 {py:attr}`~autogen_agentchat.base.ChatAgent.description`을 포함해야 합니다.

### 종료 조건

두 가지 종료 조건을 사용해 보겠습니다:
계획 에이전트가 "TERMINATE"를 보낼 때 대화를 종료하는 {py:class}`~autogen_agentchat.conditions.TextMentionTermination`과 무한 루프를 방지하기 위해 대화를 25개 메시지로 제한하는 {py:class}`~autogen_agentchat.conditions.MaxMessageTermination`입니다.

In [4]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

### 선택자 프롬프트

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 대화 컨텍스트를 기반으로 다음 발언자를 선택하기 위해 모델을 사용합니다.
워크플로우와 적절히 정렬하기 위해 사용자 정의 선택자 프롬프트를 사용하겠습니다.

In [5]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

선택자 프롬프트에서 사용할 수 있는 문자열 변수들:
- `{participants}`: 선택 후보의 이름들. 형식은 `["<이름1>", "<이름2>", ...]`입니다.
- `{roles}`: 후보 에이전트의 이름과 설명을 줄바꿈으로 구분한 목록. 각 줄의 형식은: `"<이름> : <설명>"`입니다.
- `{history}`: 이름과 메시지 내용의 이중 줄바꿈으로 구분하여 형식화된 대화 기록. 각 메시지의 형식은: `"<이름> : <메시지 내용>"`입니다.

```{tip}
선택자 프롬프트에서 모델에 너무 많은 지시사항을 주지 않도록 하세요.

너무 많다는 것은 무엇인가요? 사용하는 모델의 기능에 따라 다릅니다.
GPT-4o와 동급 모델의 경우, 각 발언자가 언제 선택되어야 하는지에 대한 조건이 있는 선택자 프롬프트를 사용할 수 있습니다.
Phi-4와 같은 더 작은 모델의 경우, 이 예제에서 사용된 것과 같이 선택자 프롬프트를 가능한 한 간단하게 유지해야 합니다.

일반적으로 각 에이전트에 대해 여러 조건을 작성하고 있다면, 사용자 정의 선택 함수를 사용하거나 작업을 별도의 에이전트나 팀에서 처리할 더 작은 순차적 작업으로 나누는 것을 고려해야 한다는 신호입니다.
```

### 팀 실행

에이전트, 종료 조건, 사용자 정의 선택자 프롬프트로 팀을 생성해 보겠습니다.

In [6]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

이제 NBA 선수에 대한 정보를 찾는 작업으로 팀을 실행합니다.

In [7]:
task = "2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?"

In [8]:
# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
먼저, 요청하신 정보를 얻기 위해 두 가지 작업으로 나누겠습니다.

1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수를 검색합니다.
2. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드를 비교하여 변화율을 계산합니다.

이 작업이 완료되면 결과를 요약하겠습니다.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_UBAzc3Zode5boLT8haAGd2Ez', arguments='{"query": "2006-2007 Miami Heat highest scorer"}', name='search_web_tool'), FunctionCall(id='call_ezEXARMfXk7eywJGVNLGrQaD', arguments='{"query": "Miami Heat total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_AwCL4wgIweQp7cEpHGj3fsXY', arguments='{"query": "Miami Heat total rebounds 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='2006-2007 시즌 마이애미 히트 선수들이 득점한 총 

TaskResult(messages=[TextMessage(id='9b80252e-1d3e-4249-b1ea-d8d7c45e0605', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 19, 176307, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='c0081d4f-fcfc-493f-b822-4f337ef954e5', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=101), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 21, 562417, tzinfo=datetime.timezone.utc), content='먼저, 요청하신 정보를 얻기 위해 두 가지 작업으로 나누겠습니다.\n\n1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수를 검색합니다.\n2. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드를 비교하여 변화율을 계산합니다.\n\n이 작업이 완료되면 결과를 요약하겠습니다.', type='TextMessage'), ToolCallRequestEvent(id='ac134c26-a262-4344-90c9-6605db437303', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=256, completion_tokens=90), metada

보시다시피, 웹 검색 에이전트가 필요한 검색을 수행하고 데이터 분석 에이전트가 필요한 계산을 완료한 후, 드웨인 웨이드가 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 한 선수이며, 2007-2008 시즌과 2008-2009 시즌 사이 그의 총 리바운드 변화율이 85.98%라는 것을 발견했습니다!

## 사용자 정의 선택자 함수

종종 선택 과정에 대한 더 나은 제어를 원합니다.
이를 위해 기본 모델 기반 선택을 재정의하는 사용자 정의 선택자 함수로 `selector_func` 인수를 설정할 수 있습니다.
이를 통해 더 복잡한 선택 로직과 상태 기반 전환을 구현할 수 있습니다.

예를 들어, 전문 에이전트 이후에 진행 상황을 확인하기 위해 계획 에이전트가 즉시 발언하기를 원합니다.

```{note}
사용자 정의 선택자 함수에서 `None`을 반환하면 기본 모델 기반 선택을 사용합니다.
``` 

```{note}
사용자 정의 선택자 함수는 SelectorGroupChat 팀에서 `.dump_component()`가 호출될 때 [직렬화](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/serialize-components.html)되지 않습니다. 사용자 정의 선택자 함수가 있는 팀 구성을 직렬화해야 하는 경우, 사용자 정의 워크플로우와 직렬화 로직을 구현하는 것을 고려하세요.
```

In [9]:
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수의 이름을 검색하라.
2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 통계를 검색하라.
3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화를 계산하라. 

TERMINATE


TaskResult(messages=[TextMessage(id='f94bf3dc-53b8-4b23-bb4a-c688b6491fc4', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 29, 312640, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='d0ed9279-f104-44d6-99f8-7d07db3a1ee0', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=108), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 30, 872310, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수의 이름을 검색하라.\n2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 통계를 검색하라.\n3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화를 계산하라. \n\nTERMINATE', type='TextMessage')], stop_reason="Text 'TERMINATE' mentioned")

대화 로그에서 계획 에이전트가 항상 전문 에이전트 이후에 즉시 발언하는 것을 볼 수 있습니다.

```{tip}
각 참가자 에이전트는 각 차례에 한 단계만 수행합니다 (도구 실행, 응답 생성 등).
{py:class}`~autogen_agentchat.agents.AssistantAgent`가 실행해야 하는 모든 도구 실행을 마쳤을 때 {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage` 반환을 중지할 때까지 반복하기를 원한다면, 마지막 메시지를 확인하고 그것이 {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage`인 경우 에이전트를 반환하여 그렇게 할 수 있습니다.
```

## 사용자 정의 후보 함수

또 다른 가능한 요구사항은 필터링된 에이전트 목록에서 다음 발언자를 자동으로 선택하는 것일 수 있습니다.
이를 위해 그룹 채팅의 각 차례에서 발언자 선택을 위한 잠재적 에이전트 목록을 필터링하는 사용자 정의 후보 함수로 `candidate_func` 매개변수를 설정할 수 있습니다.

이를 통해 주어진 에이전트 이후에 특정 에이전트 세트로 발언자 선택을 제한할 수 있습니다.

```{note}
`candidate_func`는 `selector_func`가 설정되지 않은 경우에만 유효합니다.
사용자 정의 후보 함수에서 `None` 또는 빈 목록 `[]`을 반환하면 `ValueError`가 발생합니다.
```

In [10]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitly asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
이 과제를 두 가지 주요 부분으로 나눌 수 있습니다. 첫 번째 부분은 2006-2007 시즌에 마이애미 히트의 최고 득점 선수를 찾는 것이고, 두 번째 부분은 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율을 계산하는 것입니다. 아래와 같이 각 과제를 위임하겠습니다.

1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수를 찾아라.
2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 데이터를 검색하라.
3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율을 계산하라.

지금 모든 작업이 완료되면 결과를 요약하겠습니다.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_0Mpv0Zd8S0uMX3dP52v4sBrV', arguments='{"query": "Miami Heat top scorer 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_ZsG0in3OIirIENagOJyhCkB4', arguments='{"query": "Miami Heat total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_7gXEa93CHDkvDNJYsf0lyuhG', arguments='{"query": "Miami Heat 

TaskResult(messages=[TextMessage(id='fc266e37-75d6-4db4-add2-0b56eaa80c69', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 35, 200837, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='493d121b-2ed9-4552-abee-6b54d71d0b3e', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=204), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 37, 437057, tzinfo=datetime.timezone.utc), content='이 과제를 두 가지 주요 부분으로 나눌 수 있습니다. 첫 번째 부분은 2006-2007 시즌에 마이애미 히트의 최고 득점 선수를 찾는 것이고, 두 번째 부분은 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율을 계산하는 것입니다. 아래와 같이 각 과제를 위임하겠습니다.\n\n1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수를 찾아라.\n2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 데이터를 검색하라.\n3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율을 계산하라.\n\n지금 모든 작업이 완료되면 결과를 요약하겠습니다.

대화 로그에서 웹 검색 에이전트와 데이터 분석 에이전트가 차례를 가진 후 계획 에이전트가 대화로 돌아가는 것을 볼 수 있으며, 예상대로 작업이 완료되지 않았다는 것을 발견하여 웹 검색 에이전트를 다시 호출하여 리바운드 값을 얻고 데이터 분석 에이전트를 호출하여 백분율 변화를 구했습니다.

## 사용자 피드백

실행 중 사용자 피드백을 제공하기 위해 팀에 {py:class}`~autogen_agentchat.agents.UserProxyAgent`를 추가할 수 있습니다.
{py:class}`~autogen_agentchat.agents.UserProxyAgent`에 대한 자세한 내용은 [휴먼 인 더 루프](./tutorial/human-in-the-loop.ipynb)를 참조하세요.

웹 검색 예제에서 {py:class}`~autogen_agentchat.agents.UserProxyAgent`를 사용하려면 단순히 팀에 추가하고 계획 에이전트가 발언한 후 항상 사용자 피드백을 확인하도록 선택자 함수를 업데이트하면 됩니다.
사용자가 `"APPROVE"`로 응답하면 대화가 계속되고, 그렇지 않으면 사용자가 승인할 때까지 계획 에이전트가 다시 시도합니다.

In [11]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
우선, 요청하신 내용을 두 개의 주요 작업으로 나눌 수 있습니다.

1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수의 정보를 검색하라.
2. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 수치를 비교하여 변화율을 계산하라.

이렇게 작업을 분배하겠습니다. 각 에이전트가 작업을 완료한 후 정보를 수집하여 결과를 요약하겠습니다.
---------- TextMessage (UserProxyAgent) ----------
39
---------- TextMessage (PlanningAgent) ----------
WebSearchAgent와 DataAnalystAgent의 작업을 통해 필요한 데이터를 수집하겠습니다.

1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수의 정보를 검색하라.
2. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 수치를 비교하여 변화율을 계산하라.

제공하신 정보와 함께 작업을 진행합니다. 작업이 완료되면 결과를 요약하겠습니다. TERMINATE.


TaskResult(messages=[TextMessage(id='a0eac0df-c3ec-4303-bfd5-57cb728ca0c0', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 47, 632704, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='594ef3dc-4c00-483f-bcc4-d48083d657bb', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=125), metadata={}, created_at=datetime.datetime(2025, 11, 1, 10, 14, 49, 133795, tzinfo=datetime.timezone.utc), content='우선, 요청하신 내용을 두 개의 주요 작업으로 나눌 수 있습니다.\n\n1. WebSearchAgent : 2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수의 정보를 검색하라.\n2. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 수치를 비교하여 변화율을 계산하라.\n\n이렇게 작업을 분배하겠습니다. 각 에이전트가 작업을 완료한 후 정보를 수집하여 결과를 요약하겠습니다.', type='TextMessage'), UserInputRequestedEvent(id='a4d41cf1-8675-485f-bddb-1a4d34f1af9c', source='UserProxyAgent', models_usage=None, metadata={}, c

이제 사용자의 피드백이 대화 흐름에 통합되고, 사용자가 계획 에이전트의 결정을 승인하거나 거부할 수 있습니다.

---

# 종합 실습 과제: 스마트 콘텐츠 제작 AI 팀 구축하기

지금까지 학습한 `SelectorGroupChat`의 핵심 개념들을 활용하여 **실제 콘텐츠 제작 워크플로우**를 자동화하는 다중 에이전트 시스템을 구축해보겠습니다.

## 📋 실습 목표

**블로그 포스트 자동 제작 시스템** 구현:
1. **📖 연구 에이전트**: 주제에 대한 정보 수집 및 연구
2. **✍️ 작성 에이전트**: 수집된 정보를 바탕으로 블로그 포스트 작성
3. **📝 편집 에이전트**: 작성된 콘텐츠의 품질 검토 및 개선 제안
4. **👤 승인 에이전트**: 최종 사용자 승인 과정 관리

## 🎯 학습 포인트
- **동적 에이전트 선택**: 작업 상황에 따른 최적 에이전트 자동 선택
- **커스텀 선택자 함수**: 복잡한 워크플로우 제어 로직 구현
- **Human-in-the-Loop**: 사용자 피드백 통합으로 품질 보장
- **실무 시나리오**: 실제 비즈니스에 적용 가능한 AI 시스템 설계

In [1]:
# ============================================================================
# 🎯 종합 실습: 스마트 콘텐츠 제작 AI 팀 구현
# ============================================================================

# 1. 콘텐츠 제작용 도구 함수들 정의
def research_topic(topic: str) -> str:
    """주제에 대한 연구 및 정보 수집 (Mock API)"""
    research_db = {
        "AI": """
        인공지능(AI) 연구 자료:
        - 2024년 AI 시장 규모: 1,847억 달러
        - 주요 트렌드: 생성형 AI, 멀티모달 AI, 엣지 AI
        - 응용 분야: 헬스케어, 금융, 교육, 자율주행
        - 핵심 기술: 트랜스포머, 강화학습, 컴퓨터 비전
        """,
        "파이썬": """
        Python 프로그래밍 연구 자료:
        - Python 3.12 최신 기능: 향상된 성능, 새로운 typing 기능
        - 인기 라이브러리: FastAPI, Pydantic, Pandas, NumPy
        - 활용 분야: 웹 개발, 데이터 과학, AI/ML, 자동화
        - 학습 리소스: 공식 문서, 온라인 코스, 오픈소스 프로젝트
        """,
        "클라우드": """
        클라우드 컴퓨팅 연구 자료:
        - 주요 제공업체: AWS, Azure, Google Cloud
        - 핵심 서비스: 컴퓨팅, 스토리지, 네트워킹, AI/ML
        - 트렌드: 서버리스, 컨테이너화, 멀티클라우드
        - 보안: IAM, 암호화, 컴플라이언스
        """
    }
    return research_db.get(topic, f"{topic}에 대한 연구 자료를 수집 중입니다...")

def check_content_quality(content: str) -> str:
    """콘텐츠 품질 평가 및 개선 제안"""
    word_count = len(content.split())
    
    if word_count < 50:
        return f"콘텐츠가 너무 짧습니다 ({word_count}단어). 최소 100단어 이상 작성을 권장합니다."
    elif word_count < 100:
        return f"콘텐츠 길이가 적당합니다 ({word_count}단어). 구체적인 예시나 사례를 추가하면 더 좋겠습니다."
    else:
        return f"훌륭한 콘텐츠입니다 ({word_count}단어). 가독성과 구조가 잘 갖춰져 있습니다."

def format_blog_post(title: str, content: str) -> str:
    """블로그 포스트 최종 포맷팅"""
    formatted = f"""
# {title}

{content}

---
*이 포스트는 AI 다중 에이전트 시스템으로 제작되었습니다.*
"""
    return formatted

In [2]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

# Azure OpenAI 클라이언트 설정 (실습용)
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Azure OpenAI 클라이언트 생성
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
    model=deployment_name,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    api_key=api_key,
)

model_client = azure_openai_chat_completion_client

✅ Azure OpenAI 클라이언트가 성공적으로 설정되었습니다!


In [4]:
# 2. 콘텐츠 제작 전문 에이전트들 생성

# 연구 에이전트 - 주제 조사 및 정보 수집
research_agent = AssistantAgent(
    "ResearchAgent",
    description="An agent that researches topics and gathers information for content creation.",
    model_client=model_client,
    tools=[research_topic],
    system_message="""
    당신은 전문 연구원입니다. 
    - 주어진 주제에 대해 깊이 있게 조사하고 신뢰할 수 있는 정보를 수집하세요
    - 수집한 정보를 체계적으로 정리하여 제공하세요
    - 연구 완료 후 "연구 완료"라고 알려주세요
    """,
)

# 작성 에이전트 - 블로그 포스트 작성
writer_agent = AssistantAgent(
    "WriterAgent", 
    description="An agent that writes engaging blog posts based on research data.",
    model_client=model_client,
    system_message="""
    당신은 전문 콘텐츠 작가입니다.
    - 연구 자료를 바탕으로 흥미롭고 유용한 블로그 포스트를 작성하세요
    - 독자가 쉽게 이해할 수 있도록 명확하고 논리적으로 구성하세요
    - 작성 완료 후 "작성 완료"라고 알려주세요
    """,
)

# 편집 에이전트 - 품질 검토 및 개선 제안
editor_agent = AssistantAgent(
    "EditorAgent",
    description="An agent that reviews and improves written content quality.",
    model_client=model_client,
    tools=[check_content_quality, format_blog_post],
    system_message="""
    당신은 전문 편집자입니다.
    - 작성된 콘텐츠의 품질을 객관적으로 평가하세요
    - 개선이 필요한 부분에 대해 구체적인 제안을 하세요
    - 최종 승인이 가능하다면 "편집 승인"이라고 알려주세요
    """,
)

# 사용자 승인 에이전트
approval_agent = UserProxyAgent(
    "ApprovalAgent",
    description="A human reviewer who gives final approval for content publication.",
)

In [ ]:
# 3. 고급 워크플로우 제어를 위한 커스텀 선택자 함수

def content_workflow_selector(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    """콘텐츠 제작 워크플로우에 최적화된 선택자 함수"""
    
    if not messages:
        return research_agent.name
    
    last_message = messages[-1]
    last_source = last_message.source
    
    # 1. 시작: 연구 에이전트부터
    if last_source == "user":
        return research_agent.name
    
    # 2. 연구 완료 → 작성 에이전트
    if last_source == research_agent.name and "연구 완료" in last_message.content:  # type: ignore
        return writer_agent.name
    
    # 3. 작성 완료 → 편집 에이전트  
    if last_source == writer_agent.name and "작성 완료" in last_message.content:  # type: ignore
        return editor_agent.name
    
    # 4. 편집 승인 → 사용자 승인
    if last_source == editor_agent.name and "편집 승인" in last_message.content:  # type: ignore
        return approval_agent.name
    
    # 5. 편집자가 개선 요청 → 작성자에게 재작성 요청
    if last_source == editor_agent.name and "개선" in last_message.content:  # type: ignore
        return writer_agent.name
    
    # 6. 사용자 승인 완료 → 종료
    if last_source == approval_agent.name and "승인" in last_message.content.lower():  # type: ignore
        return None  # 워크플로우 종료
    
    # 7. 사용자 거부 → 편집자가 재검토
    if last_source == approval_agent.name:
        return editor_agent.name
    
    # 기본값: 모델 기반 선택
    return None

# 4. 콘텐츠 제작팀 구성
content_termination = TextMentionTermination("PUBLISH") | MaxMessageTermination(20)

content_creation_team = SelectorGroupChat(
    [research_agent, writer_agent, editor_agent, approval_agent],
    model_client=model_client,
    termination_condition=content_termination,
    selector_func=content_workflow_selector,
    allow_repeated_speaker=True,
)

## 🧪 실습 시나리오: 블로그 포스트 제작 워크플로우 테스트

이제 구축한 콘텐츠 제작 AI 팀으로 실제 블로그 포스트를 제작해보겠습니다!

In [6]:
# 테스트 시나리오 1: AI 기술 트렌드에 대한 블로그 포스트 작성
print("=== 🤖 AI 블로그 포스트 제작 워크플로우 ===")
content_task = "AI 기술의 최신 트렌드와 미래 전망에 대한 블로그 포스트를 작성해주세요."

await Console(content_creation_team.run_stream(task=content_task))

=== 🤖 AI 블로그 포스트 제작 워크플로우 ===
---------- TextMessage (user) ----------
AI 기술의 최신 트렌드와 미래 전망에 대한 블로그 포스트를 작성해주세요.
AI 기술의 최신 트렌드와 미래 전망에 대한 블로그 포스트를 작성해주세요.
---------- ToolCallRequestEvent (ResearchAgent) ----------
[FunctionCall(id='call_QEGYjOF2kI7cOTakjdgYgMhB', arguments='{"topic":"AI 기술의 최신 트렌드와 미래 전망"}', name='research_topic')]
---------- ToolCallExecutionEvent (ResearchAgent) ----------
[FunctionExecutionResult(content='AI 기술의 최신 트렌드와 미래 전망에 대한 연구 자료를 수집 중입니다...', name='research_topic', call_id='call_QEGYjOF2kI7cOTakjdgYgMhB', is_error=False)]
---------- ToolCallSummaryMessage (ResearchAgent) ----------
AI 기술의 최신 트렌드와 미래 전망에 대한 연구 자료를 수집 중입니다...
---------- ToolCallRequestEvent (ResearchAgent) ----------
[FunctionCall(id='call_QEGYjOF2kI7cOTakjdgYgMhB', arguments='{"topic":"AI 기술의 최신 트렌드와 미래 전망"}', name='research_topic')]
---------- ToolCallExecutionEvent (ResearchAgent) ----------
[FunctionExecutionResult(content='AI 기술의 최신 트렌드와 미래 전망에 대한 연구 자료를 수집 중입니다...', name='research_topic',

TaskResult(messages=[TextMessage(id='0793017c-9b7d-4e74-bebd-48dfab30363d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 3, 3, 638406, tzinfo=datetime.timezone.utc), content='AI 기술의 최신 트렌드와 미래 전망에 대한 블로그 포스트를 작성해주세요.', type='TextMessage'), ToolCallRequestEvent(id='8fec1535-77ef-463b-b503-9eefc1b090e3', source='ResearchAgent', models_usage=RequestUsage(prompt_tokens=139, completion_tokens=25), metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 3, 4, 638771, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_QEGYjOF2kI7cOTakjdgYgMhB', arguments='{"topic":"AI 기술의 최신 트렌드와 미래 전망"}', name='research_topic')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='eff488df-a276-47e7-837d-722415be58d2', source='ResearchAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 3, 4, 640504, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='AI 기술의 최신 트렌드와 미래 전망에 대한 연구 자료를 수집 중입니

In [7]:
# 팀 리셋 후 다른 주제로 테스트
await content_creation_team.reset()

print("\n=== 🐍 Python 프로그래밍 블로그 포스트 제작 ===")
python_task = "Python 프로그래밍 초보자를 위한 실용적인 가이드 블로그 포스트를 작성해주세요."

await Console(content_creation_team.run_stream(task=python_task))


=== 🐍 Python 프로그래밍 블로그 포스트 제작 ===
---------- TextMessage (user) ----------
Python 프로그래밍 초보자를 위한 실용적인 가이드 블로그 포스트를 작성해주세요.
Python 프로그래밍 초보자를 위한 실용적인 가이드 블로그 포스트를 작성해주세요.
---------- ToolCallRequestEvent (ResearchAgent) ----------
[FunctionCall(id='call_50Uilgf8pcUatAMvBPZgjaea', arguments='{"topic":"Python programming guide for beginners"}', name='research_topic')]
---------- ToolCallExecutionEvent (ResearchAgent) ----------
[FunctionExecutionResult(content='Python programming guide for beginners에 대한 연구 자료를 수집 중입니다...', name='research_topic', call_id='call_50Uilgf8pcUatAMvBPZgjaea', is_error=False)]
---------- ToolCallSummaryMessage (ResearchAgent) ----------
---------- ToolCallRequestEvent (ResearchAgent) ----------
[FunctionCall(id='call_50Uilgf8pcUatAMvBPZgjaea', arguments='{"topic":"Python programming guide for beginners"}', name='research_topic')]
---------- ToolCallExecutionEvent (ResearchAgent) ----------
[FunctionExecutionResult(content='Python programming guide for beginners에 대

TaskResult(messages=[TextMessage(id='246d8949-fcc1-4a55-a7f8-33ac432337c2', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 5, 28, 213759, tzinfo=datetime.timezone.utc), content='Python 프로그래밍 초보자를 위한 실용적인 가이드 블로그 포스트를 작성해주세요.', type='TextMessage'), ToolCallRequestEvent(id='cbf45f29-286e-4781-a1ad-cc2497531d54', source='ResearchAgent', models_usage=RequestUsage(prompt_tokens=140, completion_tokens=20), metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 5, 28, 869461, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_50Uilgf8pcUatAMvBPZgjaea', arguments='{"topic":"Python programming guide for beginners"}', name='research_topic')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='224e1787-235c-4f48-8987-ff1c3405ad26', source='ResearchAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 1, 11, 5, 28, 872013, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='Python progr

## 🎓 실습 과제 및 확장 아이디어

### 📚 이번 종합 실습에서 배운 핵심 개념:

#### 1. **고급 워크플로우 설계**
- 복잡한 비즈니스 프로세스를 AI 에이전트로 자동화
- 각 단계별 전문 에이전트 역할 분담
- 순차적/병렬적 작업 흐름 제어

#### 2. **커스텀 선택자 함수 마스터링**
- 조건부 에이전트 선택 로직 구현
- 상태 기반 워크플로우 전환 관리
- 사용자 피드백 통합 방법

#### 3. **실무 적용 가능한 AI 시스템**
- 콘텐츠 제작, 품질 관리, 승인 프로세스
- Human-in-the-Loop 패턴으로 품질 보장
- 확장 가능한 아키텍처 설계

### 🚀 추가 도전 과제:

#### 📝 **Level 1: 기능 확장**
1. **새로운 에이전트 추가**:
   - SEO 최적화 에이전트
   - 이미지 생성 에이전트  
   - 소셜미디어 배포 에이전트

2. **고급 도구 통합**:
   - 실제 웹 검색 API 연동
   - 이미지 생성 API (DALL-E, Midjourney)
   - 번역 API로 다국어 콘텐츠

#### 🏗️ **Level 2: 아키텍처 개선**
1. **병렬 처리 도입**:
   - 다중 작성자가 동시에 섹션별 작성
   - 병렬 품질 검토 시스템

2. **메모리 및 상태 관리**:
   - 장기 기억을 위한 벡터 DB 연동
   - 프로젝트별 컨텍스트 관리

#### 🌐 **Level 3: 프로덕션 배포**  
1. **웹 서비스화**:
   - FastAPI로 REST API 구축
   - React/Vue.js 프론트엔드 개발

2. **확장성 및 안정성**:
   - 큐 시스템 도입 (Celery, Redis)
   - 모니터링 및 로깅 시스템
   - A/B 테스트를 위한 다중 에이전트 전략

### 💡 **실무 응용 아이디어**:
- **마케팅 자동화**: 캠페인 기획 → 콘텐츠 제작 → 성과 분석
- **교육 콘텐츠**: 커리큘럼 설계 → 강의 자료 제작 → 퀴즈 생성  
- **고객 지원**: 문의 분석 → 답변 생성 → 품질 검토 → 자동 응답
- **연구 보고서**: 문헌 조사 → 데이터 분석 → 보고서 작성 → 피어 리뷰

---

## 🔥 보너스 실습: 다양한 선택자 패턴 실험

여러분만의 창의적인 선택자 함수를 만들어보세요!

```python
# 예시: 라운드로빈 + 조건부 선택 혼합
def hybrid_selector(messages):
    # 특정 조건에서는 고정 순서, 나머지는 모델 선택
    pass

# 예시: 에이전트 성과 기반 선택  
def performance_based_selector(messages):
    # 이전 작업 품질에 따라 에이전트 우선순위 조정
    pass

# 예시: 시간 기반 선택
def time_aware_selector(messages):
    # 작업 시간에 따라 적절한 에이전트 선택
    pass
```

### 🎯 핵심 성공 요소:
1. **명확한 책임 분리**: 각 에이전트의 역할을 명확히 정의
2. **효율적인 선택 로직**: 불필요한 전환을 최소화
3. **사용자 경험 고려**: 투명하고 예측 가능한 워크플로우
4. **확장 가능성**: 새로운 에이전트 추가가 용이한 설계

**이제 여러분만의 혁신적인 다중 에이전트 시스템을 구축해보세요!** 🚀✨